In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [4]:
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Pandas version: 2.1.4
NumPy version: 1.26.2


## Load Orders Dataset

In [13]:
import os

In [14]:
# Correct path - note the space and (29) in folder name
data_path = "../data/raw/archive (29)/"

In [15]:
if os.path.exists(data_path):
    files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
    
    if files:
        print(f" Found {len(files)} CSV files:\n")
        print("-" * 80)
        for i, file in enumerate(sorted(files), 1):
            file_path = os.path.join(data_path, file)
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convert to MB
            print(f"{i:2d}. {file:50s} {file_size:>8.2f} MB")
        print("-" * 80)
    else:
        print(" No CSV files found in the archive folder.")
else:
    print(f" Path not found: {data_path}")

 Found 9 CSV files:

--------------------------------------------------------------------------------
 1. olist_customers_dataset.csv                            8.62 MB
 2. olist_geolocation_dataset.csv                         58.44 MB
 3. olist_order_items_dataset.csv                         14.72 MB
 4. olist_order_payments_dataset.csv                       5.51 MB
 5. olist_order_reviews_dataset.csv                       13.78 MB
 6. olist_orders_dataset.csv                              16.84 MB
 7. olist_products_dataset.csv                             2.27 MB
 8. olist_sellers_dataset.csv                              0.17 MB
 9. product_category_name_translation.csv                  0.00 MB
--------------------------------------------------------------------------------


## Load the Main Orders Dataset

In [16]:
#Load Orders Dataset
data_path = "../data/raw/archive (29)/"

In [17]:
# Load orders
orders = pd.read_csv(data_path + "olist_orders_dataset.csv")

print("Orders Dataset Loaded Successfully!")
print(f"Total Orders: {len(orders):,}")
print(f"Columns: {len(orders.columns)}")
print(f"\nDataset Shape: {orders.shape}")

Orders Dataset Loaded Successfully!
Total Orders: 99,441
Columns: 8

Dataset Shape: (99441, 8)


## First Look at Orders Data

In [18]:
# Examine Orders Structure
print("Column Names and Data Types:")
print("=" * 80)
print(orders.dtypes)
print("\n" + "=" * 80)
print("\nFirst 5 Rows:")
print("=" * 80)
print(orders.head())

Column Names and Data Types:
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object


First 5 Rows:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03

## Basic Data Quality Check

In [19]:
#  Data Quality Assessment
print("DATA QUALITY REPORT - ORDERS")
print("=" * 80)

# Check for missing values
print("\nMissing Values:")
print("-" * 80)
missing = orders.isnull().sum()
missing_pct = (orders.isnull().sum() / len(orders)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percentage': missing_pct.values
})


DATA QUALITY REPORT - ORDERS

Missing Values:
--------------------------------------------------------------------------------


In [20]:
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
print(missing_df.to_string(index=False))



                       Column  Missing_Count  Missing_Percentage
order_delivered_customer_date           2965                2.98
 order_delivered_carrier_date           1783                1.79
            order_approved_at            160                0.16


In [21]:
# Check for duplicates
print(f"\n\nDuplicate Orders: {orders.duplicated(subset=['order_id']).sum()}")

# Date range
print("\n\nDate Range:")
print("-" * 80)
print(f"Earliest Order: {orders['order_purchase_timestamp'].min()}")
print(f"Latest Order: {orders['order_purchase_timestamp'].max()}")



Duplicate Orders: 0


Date Range:
--------------------------------------------------------------------------------
Earliest Order: 2016-09-04 21:15:19
Latest Order: 2018-10-17 17:30:18


### Key Findings

- **99,441 orders** recorded from **September 2016 to October 2018** (roughly two years of data).
- **No duplicate orders** detected — the dataset is already clean on that front.
- **Missing delivery dates:** about **2.98%** of rows. These likely represent orders that were canceled or still processing.
- **Date columns:** all stored as **text strings**, so they’ll need to be **converted to datetime** before any time-based analysis.


## Convert Date Columns to Datetime

In [22]:
# Convert date columns to proper datetime format
print("Converting date columns to datetime format...")
print("-" * 80)

date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col])
    print(f"Converted: {col}")

print("\nVerifying conversion:")
print("-" * 80)
print(orders.dtypes)

print("\nDate Range After Conversion:")
print("-" * 80)
print(f"Earliest Order: {orders['order_purchase_timestamp'].min()}")
print(f"Latest Order: {orders['order_purchase_timestamp'].max()}")
print(f"Total Time Period: {(orders['order_purchase_timestamp'].max() - orders['order_purchase_timestamp'].min()).days} days")

Converting date columns to datetime format...
--------------------------------------------------------------------------------
Converted: order_purchase_timestamp
Converted: order_approved_at
Converted: order_delivered_carrier_date
Converted: order_delivered_customer_date
Converted: order_estimated_delivery_date

Verifying conversion:
--------------------------------------------------------------------------------
order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

Date Range After Conversion:
--------------------------------------------------------------------------------
Earliest Order: 2016-09-04 21:15:19
Latest Order: 2018-10-17 17:30:18
Tot

## Load Order Items Dataset

In [23]:
#  Load Order Items Dataset
order_items = pd.read_csv(data_path + "olist_order_items_dataset.csv")

print("Order Items Dataset Loaded Successfully!")
print(f"Total Order Items: {len(order_items):,}")
print(f"Columns: {len(order_items.columns)}")
print(f"Dataset Shape: {order_items.shape}")

print("\nColumn Names:")
print("-" * 80)
print(order_items.columns.tolist())

print("\nFirst 5 Rows:")
print("-" * 80)
print(order_items.head())

Order Items Dataset Loaded Successfully!
Total Order Items: 112,650
Columns: 7
Dataset Shape: (112650, 7)

Column Names:
--------------------------------------------------------------------------------
['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

First 5 Rows:
--------------------------------------------------------------------------------
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242a

## Load Payments Dataset

In [24]:
# Load Order Payments Dataset
payments = pd.read_csv(data_path + "olist_order_payments_dataset.csv")

print("Order Payments Dataset Loaded Successfully!")
print(f"Total Payment Records: {len(payments):,}")
print(f"Columns: {len(payments.columns)}")
print(f"Dataset Shape: {payments.shape}")

print("\nColumn Names:")
print("-" * 80)
print(payments.columns.tolist())

print("\nFirst 5 Rows:")
print("-" * 80)
print(payments.head())

print("\nPayment Types:")
print("-" * 80)
print(payments['payment_type'].value_counts())

Order Payments Dataset Loaded Successfully!
Total Payment Records: 103,886
Columns: 5
Dataset Shape: (103886, 5)

Column Names:
--------------------------------------------------------------------------------
['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']

First 5 Rows:
--------------------------------------------------------------------------------
                           order_id  payment_sequential payment_type  \
0  b81ef226f3fe1789b1e8b2acac839d17                   1  credit_card   
1  a9810da82917af2d9aefd1278f1dcfa0                   1  credit_card   
2  25e8ea4e93396b6fa0d3dd708e76c1bd                   1  credit_card   
3  ba78997921bbcdc1373bb41e913ab953                   1  credit_card   
4  42fdf880ba16b47b59251dd489d4441a                   1  credit_card   

   payment_installments  payment_value  
0                     8          99.33  
1                     1          24.39  
2                     1          65.71  
3     

## Load Products and Customers

In [25]:
#  Load Products and Customers Datasets
products = pd.read_csv(data_path + "olist_products_dataset.csv")
customers = pd.read_csv(data_path + "olist_customers_dataset.csv")

print("PRODUCTS DATASET")
print("=" * 80)
print(f"Total Products: {len(products):,}")
print(f"Columns: {products.columns.tolist()}")
print("\nFirst 3 Rows:")
print(products.head(3))

print("\n\nCUSTOMERS DATASET")
print("=" * 80)
print(f"Total Customers: {len(customers):,}")
print(f"Columns: {customers.columns.tolist()}")
print("\nFirst 3 Rows:")
print(customers.head(3))

PRODUCTS DATASET
Total Products: 32,951
Columns: ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

First 3 Rows:
                         product_id product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5            perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                 artes   
2  96bd76ec8810374ed1b65e291975717f         esporte_lazer   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                40.00                      287.00                1.00   
1                44.00                      276.00                1.00   
2                46.00                      250.00                1.00   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  
0            225.00              16.00              10.00             14.00  
1           1000.00              30.00   

### Data Summary

- **Orders:** 99,441 (covers roughly 2 years)
- **Order Items:** 112,650 (many orders contain multiple items)
- **Payment Records:** 103,886
- **Products:** 32,951 unique products
- **Customers:** 99,441 unique customers
- **Payment Methods:**
  - Credit card: **74%**
  - Boleto: **19%**
  - Voucher: **6%**
  - Debit card: **1%**


## Understanding Dataset Relationships

In [26]:
# Check if all orders have items
orders_with_items = order_items['order_id'].nunique()
print(f"\nOrders in main table: {len(orders):,}")
print(f"Orders with items: {orders_with_items:,}")
print(f"Match: {orders_with_items == len(orders)}")



Orders in main table: 99,441
Orders with items: 98,666
Match: False


In [27]:
# Check payment coverage
orders_with_payments = payments['order_id'].nunique()
print(f"\nOrders with payments: {orders_with_payments:,}")
print(f"Coverage: {(orders_with_payments/len(orders)*100):.2f}%")


Orders with payments: 99,440
Coverage: 100.00%


In [28]:
# Check customer coverage
orders_with_customers = orders['customer_id'].nunique()
print(f"\nUnique customers in orders: {orders_with_customers:,}")
print(f"Customers in customer table: {len(customers):,}")



Unique customers in orders: 99,441
Customers in customer table: 99,441


In [29]:
# Average items per order
avg_items = len(order_items) / len(orders)
print(f"\nAverage items per order: {avg_items:.2f}")

# Check for multiple payments per order
orders_multi_payment = payments.groupby('order_id').size()
print(f"\nOrders with multiple payment methods: {(orders_multi_payment > 1).sum():,}")


Average items per order: 1.13

Orders with multiple payment methods: 2,961


## Create Master Dataset (Merge All Tables)

In [30]:
# Start with orders
master = orders.copy()
print(f"Step 1: Starting with orders: {len(master):,} rows")


Step 1: Starting with orders: 99,441 rows


In [31]:
# Merge with order items
master = master.merge(order_items, on='order_id', how='left')
print(f"Step 2: After merging order items: {len(master):,} rows")


Step 2: After merging order items: 113,425 rows


In [32]:
# Merge with products
master = master.merge(products, on='product_id', how='left')
print(f"Step 3: After merging products: {len(master):,} rows")

# Merge with customers
master = master.merge(customers, on='customer_id', how='left')
print(f"Step 4: After merging customers: {len(master):,} rows")

Step 3: After merging products: 113,425 rows
Step 4: After merging customers: 113,425 rows


In [33]:
# For payments, we need to aggregate first (some orders have multiple payments)
payments_agg = payments.groupby('order_id').agg({
    'payment_type': 'first',
    'payment_value': 'sum',
    'payment_installments': 'max'
}).reset_index()

master = master.merge(payments_agg, on='order_id', how='left')
print(f"Step 5: After merging payments: {len(master):,} rows")

print("\nMaster dataset created successfully!")
print(f"Final shape: {master.shape}")
print(f"Total columns: {len(master.columns)}")

Step 5: After merging payments: 113,425 rows

Master dataset created successfully!
Final shape: (113425, 29)
Total columns: 29


## Calculate Key Business Metrics

In [34]:
# Total order value (price + freight)
master['total_order_value'] = master['price'] + master['freight_value']

In [35]:
# Delivery time in days
master['delivery_time_days'] = (
    master['order_delivered_customer_date'] - master['order_purchase_timestamp']
).dt.days


In [36]:
# Expected delivery time
master['expected_delivery_days'] = (
    master['order_estimated_delivery_date'] - master['order_purchase_timestamp']
).dt.days

In [37]:
# Was delivery late?
master['delivery_late'] = master['delivery_time_days'] > master['expected_delivery_days']


In [38]:
# Extract time components
master['order_year'] = master['order_purchase_timestamp'].dt.year
master['order_month'] = master['order_purchase_timestamp'].dt.month
master['order_month_name'] = master['order_purchase_timestamp'].dt.strftime('%Y-%m')
master['order_day_of_week'] = master['order_purchase_timestamp'].dt.day_name()


In [39]:
print("Calculated columns added successfully!")
print("\nNew columns:")
print("-" * 80)
new_cols = ['total_order_value', 'delivery_time_days', 'expected_delivery_days', 
            'delivery_late', 'order_year', 'order_month', 'order_month_name', 'order_day_of_week']
for col in new_cols:
    print(f"  - {col}")

print("\nSample of calculated metrics:")
print("-" * 80)
print(master[['order_id', 'price', 'freight_value', 'total_order_value', 
              'delivery_time_days', 'delivery_late']].head())


Calculated columns added successfully!

New columns:
--------------------------------------------------------------------------------
  - total_order_value
  - delivery_time_days
  - expected_delivery_days
  - delivery_late
  - order_year
  - order_month
  - order_month_name
  - order_day_of_week

Sample of calculated metrics:
--------------------------------------------------------------------------------
                           order_id  price  freight_value  total_order_value  \
0  e481f51cbdc54678b7cc49136f2d6af7  29.99           8.72              38.71   
1  53cdb2fc8bc7dce0b6741e2150273451 118.70          22.76             141.46   
2  47770eb9100c2d0c44946d9cf07ec65d 159.90          19.22             179.12   
3  949d5b44dbf5de918fe9c16f97b45f8a  45.00          27.20              72.20   
4  ad21c59c0840e6cb83a9ceb5573f8159  19.90           8.72              28.62   

   delivery_time_days  delivery_late  
0                8.00          False  
1               13.00          

## Save the Master Dataset

In [40]:
output_path = "../data/processed/"

# Save master dataset
master.to_csv(output_path + "master_sales_data.csv", index=False)
print(f"Master dataset saved to: {output_path}master_sales_data.csv")
print(f"File size: {master.memory_usage(deep=True).sum() / (1024**2):.2f} MB in memory")

Master dataset saved to: ../data/processed/master_sales_data.csv
File size: 124.97 MB in memory


In [41]:
# Quick summary
print("\nMaster Dataset Summary:")
print("=" * 80)
print(f"Total rows: {len(master):,}")
print(f"Total columns: {len(master.columns)}")
print(f"Date range: {master['order_purchase_timestamp'].min()} to {master['order_purchase_timestamp'].max()}")
print(f"Total revenue: ${master['total_order_value'].sum():,.2f}")


Master Dataset Summary:
Total rows: 113,425
Total columns: 37
Date range: 2016-09-04 21:15:19 to 2018-10-17 17:30:18
Total revenue: $15,843,553.24


### Important Observations

- **Total Revenue:** \$15.8 million over the 2-year period.
- **Order Items:** 113,425 (many orders include multiple products).
- **Orders Without Items:** 775 (likely canceled or invalid orders).
- **Multiple Payment Methods:** 2,961 orders used more than one payment method.
- **Average Items per Order:** 1.13


In [42]:
#Summary
print("DAY 1 - DATA LOADING AND PREPARATION - COMPLETE")
print("=" * 80)
print("\nWhat we accomplished:")
print("-" * 80)
print("1. Loaded 9 CSV files from Brazilian e-commerce dataset")
print("2. Converted date columns to proper datetime format")
print("3. Merged all datasets into master analysis table")
print("4. Added calculated business metrics")
print("5. Saved processed data for analysis")

print("\nKey Statistics:")
print("-" * 80)
print(f"Total Orders: {len(orders):,}")
print(f"Total Order Items: {len(master):,}")
print(f"Total Revenue: ${master['total_order_value'].sum():,.2f}")
print(f"Date Range: {master['order_year'].min()} - {master['order_year'].max()}")
print(f"Unique Customers: {master['customer_id'].nunique():,}")
print(f"Unique Products: {master['product_id'].nunique():,}")
print(f"States Covered: {master['customer_state'].nunique()}")

print("\nNext Steps (Day 2):")
print("-" * 80)
print("1. Revenue analysis by time period")
print("2. Customer behavior analysis")
print("3. Product category performance")
print("4. Delivery performance metrics")
print("5. Create visualizations")

print("\nFiles created:")
print("-" * 80)
print("- notebooks/01_data_exploration.ipynb")
print("- data/processed/master_sales_data.csv")

DAY 1 - DATA LOADING AND PREPARATION - COMPLETE

What we accomplished:
--------------------------------------------------------------------------------
1. Loaded 9 CSV files from Brazilian e-commerce dataset
2. Converted date columns to proper datetime format
3. Merged all datasets into master analysis table
4. Added calculated business metrics
5. Saved processed data for analysis

Key Statistics:
--------------------------------------------------------------------------------
Total Orders: 99,441
Total Order Items: 113,425
Total Revenue: $15,843,553.24
Date Range: 2016 - 2018
Unique Customers: 99,441
Unique Products: 32,951
States Covered: 27

Next Steps (Day 2):
--------------------------------------------------------------------------------
1. Revenue analysis by time period
2. Customer behavior analysis
3. Product category performance
4. Delivery performance metrics
5. Create visualizations

Files created:
----------------------------------------------------------------------------